In [47]:
# !pip install altair vega-datasets -q

In [48]:
import pandas as pd
import altair as alt 

bgu = pd.read_csv('./takeoff-merged-VQ-BGU-30s.csv', parse_dates=['reportts'])
bgu['actype'] = 'A321-neo VQ-BGU'
bdu = pd.read_csv('./takeoff-merged-VQ-BDU-30s.csv', parse_dates=['reportts'])
bdu['actype'] = 'A321-neo VQ-BDU'
bwb = pd.read_csv('./takeoff-merged-VP-BWB-30s.csv', parse_dates=['reportts'])
bwb['actype'] = 'A320-neo VP-BWB'
bcr = pd.read_csv('./takeoff-merged-VQ-BCR-30s.csv', parse_dates=['reportts'])
bcr['actype'] = 'A320-neo VQ-BCR'

dataset = pd.concat([bgu, bdu, bwb, bcr]) \
  .drop(columns=['file_path', 'reason']) \
  .sort_values('reportts')

In [49]:
def timestamp(t):
  return pd.to_datetime(t).timestamp() * 1000

In [72]:
important = ['tat', 'egt', 'naiup', 'naidp', 'fmvc', 'baf', 'fdp', 'aoc', 'oip', 'alt', 'ttp', 'bbv']

output_params = ['egtm', 'n1_delt', 'n2_delt']

input_radio = alt.binding_radio(options=important, name="Input parameter")
input_selection = alt.param(bind=input_radio, value='tat')

output_radio = alt.binding_radio(options=output_params, name="Output parameter")
output_selection = alt.param(bind=output_radio, value='egtm')

chart = alt.Chart(dataset, width=800, height=400) \
    .mark_point(filled=True) \
    .encode(
        x=alt.X('x:Q', title=''),
        y=alt.Y('y:Q', title=''),
        size=alt.value(20)
    ) \
    .transform_calculate(
        x=f'datum[{input_selection.name}]',
        y=f'datum[{output_selection.name}]',
    ) \
    .add_params(input_selection, output_selection) 

In [51]:
brush = alt.selection_interval(encodings=['x'])

important = ['nait', 'tn', 'tfuel', 'tat', 'egt']

parameter_radio = alt.binding_radio(options=important, name="Parameter")
parameter_selection = alt.param(bind=parameter_radio, value="tat")

time_line = alt.Chart(dataset, width=800, height=300) \
    .mark_point(filled=True) \
    .encode(
        x=alt.X('reportts'),
        y=alt.Y('y:Q', scale=alt.Scale(zero=False)),
        color='pos:N',
        size=alt.value(10),
    ) \
    .transform_calculate(
        y=f'datum[{parameter_selection.name}]',
    ) \
    .add_params(parameter_selection)


upper = time_line.encode(alt.X('reportts:T').scale(domain=brush))
lower = time_line.properties(height=40).add_params(brush)

params_curve = upper & lower

In [89]:
brush = alt.selection_interval(encodings=['x'])

selection = alt.selection_point(fields=['actypepos'], bind='legend')

time_line = alt.Chart(dataset, width=600, height=400) \
    .transform_calculate(
        actypepos="datum.actype + ' / ' + datum.pos"
    ) \
    .mark_point(filled=True) \
    .encode(
        x=alt.X('reportts'),
        y=alt.Y('egtm:Q'),
        size=alt.value(10),
        opacity=alt.condition(selection, alt.value(1), alt.value(0)),
        color=alt.Color('actypepos:N'), 
    ).add_params(selection)

upper = time_line.encode(alt.X('reportts:T').scale(domain=brush)).properties(title="EGT Margin (показатель состояния двигателя)")
lower = time_line.properties(height=30).add_params(brush).add_params(selection)

egtm_timeline = upper & lower

In [96]:
important = ['nait', 'tn', 'tfuel', 'tat', 'egt']

parameter_radio = alt.binding_radio(options=important, name="Parameter")
parameter_selection = alt.param(bind=parameter_radio, value="tat")

chart = alt.Chart(dataset, width=600, height=400) \
    .properties(title="Параметры с сезонными трендами") \
    .mark_point(filled=True) \
    .transform_calculate(
        actypepos="datum.actype + ' / ' + datum.pos"
    ) \
    .encode(
        x=alt.X('reportts:T', title=''),
        y=alt.Y('y:Q', title=''),
        color=alt.Color('actypepos:N', legend=alt.Legend(title='Номер ВС / позиция двигателя')), 
        size=alt.value(10)
    ) \
    .transform_calculate(y=f'datum[{parameter_selection.name}]').add_params(parameter_selection)

chart_regr = chart.transform_regression('reportts', 'y', method='poly', order=5)\
    .encode(size=alt.value(2)) \
    .mark_line(width=0.1)
regression = chart + chart_regr

In [65]:
airports = pd.read_csv('https://datahub.io/core/airport-codes/r/airport-codes.csv')

airports_coord =  airports[['gps_code', 'coordinates', 'name', 'municipality', 'iata_code']]
airports_coord = dataset.merge(airports_coord, 'inner', left_on='dep', right_on='gps_code')
airports_coord[['long', 'lat']] = airports_coord['coordinates'].str.split(',', expand=True)

In [155]:
from vega_datasets import data
world = data.world_110m.url

base = alt.Chart(airports_coord, width=600, height=400) \
  .transform_aggregate(dep_count='count(dep)', groupby=['dep', 'lat', 'long', 'municipality', 'iata_code']) \
  .transform_calculate(dep_size='100 + datum.dep_count', label="datum.iata_code + '(' + datum.dep_count + ')'")

map = alt.Chart(alt.topo_feature(world, 'countries'), width=600, height=400) \
  .properties(title="Карта частоты вылетов") \
  .mark_geoshape(fill='#ade9ff', stroke='#706545', opacity=0.9, strokeWidth=0.5) \
  .project(type='mercator', scale=210, translate=[70, 380])

dep_legend = alt.Legend(title='Число вылетов', values=[30, 100, 200, 400])

points = base \
  .mark_circle().encode(
    latitude='lat:Q',
    longitude='long:Q',
    size=alt.Size('dep_size:Q', legend=dep_legend, scale=alt.Scale(range=[30, 500])),
    tooltip='municipality',
    color=alt.ColorValue('#4c52fc'),
    opacity=alt.condition('datum.dep_count > 75', alt.value(1), alt.value(0.5))
  ).project(type='mercator', scale=210)

text = base.mark_text(dy=-15) \
  .encode(
    latitude='lat:Q',
    longitude='long:Q',
    text='label:N',
    opacity=alt.condition('datum.dep_count > 75', alt.value(1), alt.value(0))
  ).project(type='mercator', scale=210)

choropleth = alt.layer(map, points, text)

In [ ]:
(egtm_timeline | choropleth) & regression